In [8]:
#libaries
import pandas as pd
import pyreadr
import numpy as np
import os
import pickle

In [9]:
##########################################################################
# load city df and variables
##########################################################################
result = pyreadr.read_r('/data/users/laura.owen/extremes/heatwaves/HadUKGrid/dur-clim/coords/UK_top30_cities.Rda')
city_df = result['city_df']

yrs = list(range(1980, 2081, 2)) #51years
RPy = [0.5, 2, 5, 10, 20, 50, 100, 200, 500, 1000] #10RPlevels
do_dur = list(range(1, 10)) #9 durations
stens = ["01", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "15"] #12 ensembles

# Lists to store per-city results 
city_levxally_lists = {}
city_nDeqdally_lists = {}
city_th_lists = {}
city_mhwt_lists = {}

# Lists to store mean values across cities
city_mean_levxally = {}
city_mean_nDeqdally = {}
city_mean_th = {}
city_mean_mhwt = {}

In [10]:
# Helper function to load .rds or return fallback array
def try_read_rds(path, fallback_shape=None):
    if os.path.exists(path):
        result = pyreadr.read_r(path)
        return result[None]  # assume single unnamed object
    else:
        if fallback_shape:
            print(f"File not found: {path}, filling with NaNs")
            return np.full(fallback_shape, np.nan)
        else:
            raise FileNotFoundError(f"File not found and no fallback: {path}")

In [11]:
##########################################################################
# Loop over each city to get ensemble probs and ensemble mean probs to store
##########################################################################
for _, row in city_df.iterrows():
    city_name = str(row["city"]).replace(" ", "_")
    xco = str(row["lon_index"])
    yco = str(row["lat_index"])
    #print(city_name, xco, yco)

    # create lists to store data for each ens
    levxally_list = {}
    nDeqdally_list = {}
    th_list = {}
    mhwt_list = {}
        
    ##########################################################################
    # read in all ens prob files and store
    ##########################################################################
    for s in stens:
        base_path = f"/home/users/laura.owen/old-home/data/users/lowen/extremes/heatwaves/HadUKGrid/dur-clim/probs/{s}"
        levxally_path = f"{base_path}/levxally_{city_name}_{xco}_{yco}_{s}.rds"
        nDeqdally_path = f"{base_path}/nDeqdally_{city_name}_{xco}_{yco}_{s}.rds"
        th_path = f"{base_path}/tha_{city_name}_{xco}_{yco}_{s}.rds"
        mhwt_path = f"{base_path}/mhwt_{city_name}_{xco}_{yco}_{s}.rds"
        
        levxally_list[s] = try_read_rds(levxally_path, fallback_shape=(3, 10, 51)) #servity type (severity,mean,peakvalue), RP and years
        nDeqdally_list[s] = try_read_rds(nDeqdally_path, fallback_shape=(9, 51)) #duration and years
        th_list[s] = try_read_rds(th_path, fallback_shape=(51,))
        mhwt_list[s] = try_read_rds(mhwt_path, fallback_shape=(51,)) 

    city_levxally_lists[city_name] = levxally_list
    city_nDeqdally_lists[city_name] = nDeqdally_list
    city_th_lists[city_name] = th_list
    city_mhwt_lists[city_name] = mhwt_list

    ##########################################################################
    # read in ensemble mean prob files and store
    ##########################################################################
    mean_base = "/home/users/laura.owen/old-home/data/users/lowen/extremes/heatwaves/HadUKGrid/dur-clim/probs/meanens"
    mean_levxally_path = f"{mean_base}/levxally_{city_name}_{xco}_{yco}.rds"
    mean_nDeqdally_path = f"{mean_base}/nDeqdally_{city_name}_{xco}_{yco}.rds"
    mean_th_path = f"{mean_base}/tha_{city_name}_{xco}_{yco}.rds"
    mean_mhwt_path = f"{mean_base}/mhwt_{city_name}_{xco}_{yco}.rds"

    city_mean_levxally[city_name] = try_read_rds(mean_levxally_path, fallback_shape=(3, 10, 51))
    city_mean_nDeqdally[city_name] = try_read_rds(mean_nDeqdally_path, fallback_shape=(9, 51))
    city_mean_th[city_name] = try_read_rds(mean_th_path, fallback_shape=(51,))
    city_mean_mhwt[city_name] = try_read_rds(mean_mhwt_path, fallback_shape=(51,))


File not found: /home/users/laura.owen/old-home/data/users/lowen/extremes/heatwaves/HadUKGrid/dur-clim/probs/01/levxally_Islington_147_44_01.rds, filling with NaNs
File not found: /home/users/laura.owen/old-home/data/users/lowen/extremes/heatwaves/HadUKGrid/dur-clim/probs/01/nDeqdally_Islington_147_44_01.rds, filling with NaNs
File not found: /home/users/laura.owen/old-home/data/users/lowen/extremes/heatwaves/HadUKGrid/dur-clim/probs/01/tha_Islington_147_44_01.rds, filling with NaNs
File not found: /home/users/laura.owen/old-home/data/users/lowen/extremes/heatwaves/HadUKGrid/dur-clim/probs/01/mhwt_Islington_147_44_01.rds, filling with NaNs
File not found: /home/users/laura.owen/old-home/data/users/lowen/extremes/heatwaves/HadUKGrid/dur-clim/probs/04/levxally_Islington_147_44_04.rds, filling with NaNs
File not found: /home/users/laura.owen/old-home/data/users/lowen/extremes/heatwaves/HadUKGrid/dur-clim/probs/04/nDeqdally_Islington_147_44_04.rds, filling with NaNs
File not found: /home/u

In [ ]:
# ##########################################################################
# # save dictionary of th and mhwt
# ##########################################################################
# dataframes = {}  # Dictionary to store DataFrames for each city

# for city_row in city_df.itertuples():
#     city = str(city_row.city).replace(" ", "_")

#     th_list = city_th_lists[city]
#     mhwt_list = city_mhwt_lists[city]

#     mean_th_raw = city_mean_th[city]
#     mean_mhwt_raw = city_mean_mhwt[city]

#     # Handle fallback conversion to numpy arrays
#     if isinstance(mean_th_raw, pd.DataFrame):
#         if mean_th_raw.shape[1] == 1:
#             mean_th = mean_th_raw.iloc[:, 0].values
#         else:
#             raise ValueError(f"Unexpected shape for mean_th_raw: {mean_th_raw.shape}")
#     else:
#         mean_th = mean_th_raw
    
#     if isinstance(mean_mhwt_raw, pd.DataFrame):
#         if mean_mhwt_raw.shape[1] == 1:
#             mean_mhwt = mean_mhwt_raw.iloc[:, 0].values
#         else:
#             raise ValueError(f"Unexpected shape for mean_mhwt_raw: {mean_mhwt_raw.shape}")
#     else:
#         mean_mhwt = mean_mhwt_raw

#     data = []

#     # Add ensemble members
#     for ens in stens:
#         df_th = th_list[ens]
#         df_mhwt = mhwt_list[ens]

#         thresholds = df_th.iloc[:, 0].values if isinstance(df_th, pd.DataFrame) else df_th
#         mhwts = df_mhwt.iloc[:, 0].values if isinstance(df_mhwt, pd.DataFrame) else df_mhwt

#         for year, threshold, mhwt in zip(yrs, thresholds, mhwts):
#             data.append({
#                 "year": year,
#                 "threshold": threshold,
#                 "mhwt": mhwt,
#                 "ensemble": ens
#             })

#     # Add threshold ensemble mean 
#     if len(mean_th) == len(yrs):
#         th_years = yrs
#         th_values = mean_th
#     elif len(mean_th) == 3:
#         th_years = [1980, 2020, 2080]
#         th_values = mean_th
#     else:
#         print(f"Warning: Unexpected mean_th length for city {city} ({len(mean_th)})")
#         th_years = []
#         th_values = []

#     # Add mhwt ensemble mean 
#     if len(mean_mhwt) == len(yrs):
#         mhwt_years = yrs
#         mhwt_values = mean_mhwt
#     elif len(mean_mhwt) == 3:
#         mhwt_years = [1980, 2020, 2080]
#         mhwt_values = mean_mhwt
#     else:
#         print(f"Warning: Unexpected mean_mhwt length for city {city} ({len(mean_mhwt)})")
#         mhwt_years = []
#         mhwt_values = []

#     # Add threshold mean records
#     for year, threshold in zip(th_years, th_values):
#         data.append({
#             "year": year,
#             "threshold": threshold,
#             "mhwt": None,  # placeholder if mhwt not included
#             "ensemble": "mean"
#         })

#     # Add mhwt mean records
#     for year, mhwt in zip(mhwt_years, mhwt_values):
#         data.append({
#             "year": year,
#             "threshold": None,  # placeholder if threshold not included
#             "mhwt": mhwt,
#             "ensemble": "mean"
#         })

#     # Create DataFrame for the current city
#     df = pd.DataFrame(data)

#     dataframes[city] = df  # Store the DataFrame in the dictionary

# # Save the dataframes dictionary to a file
# with open("../dataframes/th_mhwt_dataframes_dict.pkl", "wb") as file:
#     pickle.dump(dataframes, file)


In [ ]:
##########################################################################
# save dictionary of durations
##########################################################################
duration_dataframes = {}

for city_row in city_df.itertuples():
    city = str(city_row.city).replace(" ", "_")

    nDeqdally_list = city_nDeqdally_lists[city]         # dict: {ensemble: [duration x year]}
    mean_nDeqdally = city_mean_nDeqdally[city]  # shape [duration x year]

    if mean_nDeqdally.shape != (9, len(yrs)):
        print(f"Skipping city {city}: mean_nDeqdally shape {mean_nDeqdally.shape} != (9, {len(yrs)})")
        add_mean = False
    else:
        add_mean = True

    mean_nDeqdally = mean_nDeqdally.values if isinstance(mean_nDeqdally, pd.DataFrame) else mean_nDeqdally

    data = []

    # Add ensemble members
    for ens in stens:
        df_dur = nDeqdally_list[ens]

        # Convert if it's a DataFrame
        values = df_dur.values if isinstance(df_dur, pd.DataFrame) else df_dur

        for dur in range(1, values.shape[0]+1):
            for year_idx, year in enumerate(yrs):
                data.append({
                    "city": city,
                    "duration": dur,
                    "year": year,
                    "nduration": values[dur-1, year_idx],
                    "ensemble": ens
                })

    # Add ensemble mean
    if add_mean:
        for dur in range(1, mean_nDeqdally.shape[0]+1):
            for year_idx, year in enumerate(yrs):
                data.append({
                    "city": city,
                    "duration": dur,
                    "year": year,
                    "nduration": mean_nDeqdally[dur-1, year_idx],
                    "ensemble": "mean"
                })

    # Create DataFrame for city
    df = pd.DataFrame(data)
    duration_dataframes[city] = df

# Save dictionary of duration-based data
with open("../dataframes/duration_dataframes_dict.pkl", "wb") as file:
    pickle.dump(duration_dataframes, file)


In [14]:
# ##########################################################################
# # save dictionary of levxally
# ##########################################################################
# #LEVXALLY(3,10,51) - 3 severity types, 10 RP levels, 51 years
# #1) severity 2) mean severity 3) peak value
# levxally_dataframes = {}

# for city_row in city_df.itertuples():
#     city = str(city_row.city).replace(" ", "_")

#     levxally_list = city_levxally_lists[city]  # dict: {ensemble: [severity x RP x year]}
#     mean_levxally = city_mean_levxally[city]  # shape [severity x RP x year]

#     # Check shape of mean_levxally
#     expected_shape = (3, 10, len(yrs))
#     if mean_levxally.shape != expected_shape:
#         print(f"Skipping city {city}: mean_levxally shape {mean_levxally.shape} != {expected_shape}")
#         add_mean = False
#     else:
#         add_mean = True

#     # Convert mean_levxally to numpy array if it's a DataFrame
#     mean_levxally = mean_levxally.values if isinstance(mean_levxally, pd.DataFrame) else mean_levxally

#     data = []

#     # Add ensemble members
#     for ens in stens:
#         levx = levxally_list[ens]

#         # Convert to numpy if DataFrame
#         levx_values = levx.values if isinstance(levx, pd.DataFrame) else levx

#         # Iterate over severity types, RP levels, and years
#         for sev_idx in range(levx_values.shape[0]):
#             for rp_idx in range(levx_values.shape[1]):
#                 for year_idx, year in enumerate(yrs):
#                     data.append({
#                         "city": city,
#                         "severity_type": sev_idx + 1,  # or use names if available
#                         "rp_level": rp_idx + 1,
#                         "year": year,
#                         "severity_value": levx_values[sev_idx, rp_idx, year_idx].item(),
#                         "ensemble": ens
#                     })

#     # Add ensemble mean
#     if add_mean:
#         for sev_idx in range(mean_levxally.shape[0]):
#             for rp_idx in range(mean_levxally.shape[1]):
#                 for year_idx, year in enumerate(yrs):
#                     data.append({
#                         "city": city,
#                         "severity_type": sev_idx + 1,
#                         "rp_level": rp_idx + 1,
#                         "year": year,
#                         "severity_value": mean_levxally[sev_idx, rp_idx, year_idx].item(),
#                         "ensemble": "mean"
#                     })

#     # Create DataFrame for city
#     df = pd.DataFrame(data)
#     levxally_dataframes[city] = df

# # Save dictionary of severity-based data
# with open("../dataframes/levxally_dataframes_dict.pkl", "wb") as file:
#     pickle.dump(levxally_dataframes, file)

